# Data Manipulation in Python
## Workflow
This is only a sample, very basic workflow. There will be resources at the end for material that explores this subject in a lot more depth.

There's a lot more that can be done, and a lot more tools that can be used.

1. Download CSV
1. Import data into a Pandas Dataframe
1. Clean it up
1. Extract slices of data
1. Data Visualization (next section)

## Downloading some interesting Data
Datasets can come in many formats, notably as CSVs, Excel Files, connection to databases, SQLite Database files, HP5 files and so, so many others.
We'll be using some simple CSV data.

### Our dataset
For this workshop, we'll be working with a dataset that's commonly used for these sort of tutorials. It's a database of the top 5000 movies from TMDb which is a more tolerant competitor to IMDb which doesn't send DMCA takedown requests to people who scrape their website.

The dataset is available on Kaggle at https://www.kaggle.com/tmdb/tmdb-movie-metadata

We normally would need to download the file, but I've added the file we need to our repository so that we don't need to do anything.

The commands below should work just fine on the cloud environments and on non-Windows environments.

In [ ]:
#!mkdir sources
#!curl -o sources/tmdb-5000-movie-dataset.zip https://marcolussetti.com/workshops/python4data2017/sources/tmdb-5000-movie-dataset.zip

In [ ]:
#!unzip sources/tmdb-5000-movie-dataset.zip -d sources/

In [ ]:
#!ls sources/*.csv

Now that we have a csv file downloaded in the sources directory, we can import into Pandas.

In [ ]:
import pandas as pd

In [ ]:
dataframe = pd.DataFrame.from_csv('sources/tmdb_5000_movies.csv')
dataframe

### Cleaning up the data
What are some issues you can spot with this data, by just glancing at the dataframe above?

These are the ones I could think of quickly when I prepared the presentation, but I'm sure I missed some:
- The row index is the budget, whereas it should be the id.
- There are a lot of columns, do we even need all of them?
- There are a bunch of movies empty values for some values (indicated by NaN in Pandas)
- There are movies with a budget of 0
- There are movies with revenue of 0
- The genre, keywords, production_company, production_country, language columns contain data in what appears to be JSON structures. Some even include what look like unicode escaping sequences.
- There are movies with no languages listed (denoted by an empty JSON array)
- There are movies whose rating is based on thousand of votes, and movies whose rating is based on 5 votes.

##### Fixing the index

In [ ]:
dataframe = dataframe.reset_index()
dataframe = dataframe.set_index('id')
dataframe

##### Removing columns we don't need

In [ ]:
dataframe_filtered = dataframe[['title', 'release_date', 'runtime',
                                'budget', 'revenue', 'vote_average', 'vote_count', 'genres']]
dataframe_filtered

#### Exploring data
Let's look at the what sort of data we're looking at!

In [ ]:
print("Rows", len(dataframe_filtered))

print("Avg budget", round(dataframe_filtered['budget'].mean(), 0),
      "Min budget", dataframe_filtered['budget'].min(), "Max budget", dataframe_filtered['budget'].max())

print("Avg revenue", round(dataframe_filtered['revenue'].mean(), 0),
      "Min revenue", dataframe_filtered['revenue'].min(), "Max revenue", dataframe_filtered['revenue'].max())

In [ ]:
dataframe.describe()

Now let's say we can to add a new column, called ROI that tracks the return on the investement.

The formula for ROI is: $\frac{Revenue - Cost}{Cost}$

However, we have several columns where the revenue and or cost are zero, which is obviously not very useful for our calculations. So we need to drop all such movies.

In [ ]:
has_revenue = dataframe_filtered['revenue'] > 0
has_budget = dataframe_filtered['budget'] > 0

In [ ]:
# How many have no votes though?
has_votes = dataframe_filtered['vote_count'] > 0
print("Movies with 0 votes:", len(dataframe_filtered) -
      len(dataframe_filtered[has_votes]))

has_votes = dataframe_filtered['vote_count'] >= 10
print("Movies with 10 or less votes:", len(
    dataframe_filtered) - len(dataframe_filtered[has_votes]))

In [ ]:
movies_with_revenue = dataframe_filtered[has_budget & has_revenue & has_votes]
movies_with_revenue.describe()

In [ ]:
print("Rows", len(movies_with_revenue))

print("Avg budget", round(movies_with_revenue['budget'].mean(), 0),
      "Min budget", movies_with_revenue['budget'].min(), "Max budget", movies_with_revenue['budget'].max())

print("Avg revenue", round(movies_with_revenue['revenue'].mean(), 0),
      "Min revenue", movies_with_revenue['revenue'].min(), "Max revenue", movies_with_revenue['revenue'].max())

print("Avg rating", round(movies_with_revenue['vote_average'].mean(), 0),
      "Min rating", movies_with_revenue['vote_average'].min(), "Max rating", movies_with_revenue['vote_average'].max())

print("Avg votes", round(movies_with_revenue['vote_count'].mean(), 0),
      "Min votes", movies_with_revenue['vote_count'].min(), "Max votes", movies_with_revenue['vote_count'].max())

print("Avg runtime", round(movies_with_revenue['runtime'].mean(), 0),
      "Min runtime", movies_with_revenue['runtime'].min(), "Max runtime", movies_with_revenue['runtime'].max())

In [ ]:
# Wait, what sort of movies has 338 minutes runtime?
from datetime import timedelta
print('338 minutes is equivalent to', timedelta(minutes=338), 'hours')
movies_with_revenue.loc[movies_with_revenue['runtime'].idxmax()]

Now we can create our new ROI column. Let's review the formula:

$\frac{Revenue - Cost}{Cost}$

In [ ]:
pd.options.mode.chained_assignment = None  # Disable warning we don't care about
movies_with_revenue['ROI'] = (movies_with_revenue['revenue'] -
                              movies_with_revenue['budget']) / movies_with_revenue['budget']
movies_with_revenue

In [ ]:
print("Avg ROI", round(movies_with_revenue['ROI'].mean(), 0),
      "Min ROI", movies_with_revenue['ROI'].min(), "Max ROI", movies_with_revenue['ROI'].max())

In [ ]:
movies_with_revenue.sort_values('ROI', ascending=False).head()

In [ ]:
movies_with_revenue[(movies_with_revenue['budget'] < 1000)
                    | (movies_with_revenue['revenue'] < 1000)]

In [ ]:
# Let's fix some stuff, the data is from Wikipedia
# Tarnation. This is actually semi-accurate and we can leave it as is.
# We don't have enough info from Wikipedia for both budgets & revenues
movies_to_drop = [217708, 78383, 22649, 11980]
movies_with_revenue.drop(movies_to_drop, inplace=True)

# Budgets were supposed to be in millions
budgets_to_increment = [16340, 1613, 10397, 13006, 3082]
for movie in budgets_to_increment:
    movies_with_revenue.at[movie, 'budget'] = 1000000

# Revenues were supposed to be in millions
revenues_to_increment = [16340, 1613, 10397, 13006, 14844, 2196, 18475, 10944]
for movie in revenues_to_increment:
    movies_with_revenue.at[movie, 'revenue'] *= 1000000

# Budgets are just plain wrong
# F.I.S.T.
movies_with_revenue.at[28932, 'budget'] = 8000000
movies_with_revenue.at[28932, 'revenue'] = 20388920
# I Married a Strange Person!
movies_with_revenue.at[51942, 'budget'] = 250000
movies_with_revenue.at[51942, 'revenue'] = 467272

# Check what's left
movies_with_revenue[(movies_with_revenue['budget'] < 1000)
                    | (movies_with_revenue['revenue'] < 1000)]

<center>![Tarnation](https://images-na.ssl-images-amazon.com/images/I/51X9ED5D5EL.jpg "Tarnation")</center>

In [ ]:
movies_with_revenue['ROI'] = (movies_with_revenue['revenue'] -
                              movies_with_revenue['budget']) / movies_with_revenue['budget']

print(movies_with_revenue['ROI'].describe())
print()
print(movies_with_revenue.loc[movies_with_revenue['ROI'].idxmax()])

In [ ]:
movies_with_revenue.describe()

In [ ]:
movies_with_revenue.corr(method='pearson')

In [ ]:
movies_with_revenue['release_date'] = pd.to_datetime(
    movies_with_revenue['release_date'])

movies_with_revenue['Year'] = movies_with_revenue['release_date'].dt.year
movies_with_revenue['Month'] = movies_with_revenue['release_date'].dt.month
movies_with_revenue.describe()

In [ ]:
movies_with_revenue.corr()

# Data Visualization
Now that we have some semi-useful data, we can experiment with some graphs.

We will use Seaborn and Matplotlib, the standard plotting library in Python.

In [ ]:
%matplotlib inline
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### Seaborn

In [ ]:
# Scatterplot of budget vs revenue
sns.lmplot(x="budget", y="revenue", data=movies_with_revenue)
# sns.lmplot(x="budget", y="revenue", data=movies_with_revenue, scatter_kws={"s": 2})

In [ ]:
# Same scale on both axes
sns.lmplot(x="budget", y="revenue", data=movies_with_revenue)
plt.xlim(0, max(movies_with_revenue['budget'].max(), movies_with_revenue['revenue'].max()))
plt.ylim(0, max(movies_with_revenue['budget'].max(), movies_with_revenue['revenue'].max()))
plt.gca().set_aspect('equal', adjustable='box')

In [ ]:
scaled_values = movies_with_revenue[['budget', 'revenue']].apply(lambda x: x / 1000000)

sns.lmplot(x='budget', y='revenue', data=scaled_values)
plt.xlim(0, max(scaled_values['budget'] + scaled_values['revenue']))
plt.ylim(0, max(scaled_values['budget'] + scaled_values['revenue']))
plt.gca().set_aspect('equal', adjustable='box')

In [ ]:
scaled_values = movies_with_revenue[['budget', 'revenue']].sample(n=50).apply(lambda x: x / 1000000)

sns.lmplot(x='budget', y='revenue', data=scaled_values, fit_reg=False)

In [ ]:
scaled_values = movies_with_revenue[['title', 'budget', 'revenue']].sort_values('revenue', ascending=False)[:25]
scaled_values['budget'] = scaled_values['budget'].apply(lambda x: x / 1000000)
scaled_values['revenue'] = scaled_values['revenue'].apply(lambda x: x / 1000000)
# scaled_values.head()
sns.boxplot(data=scaled_values[['revenue']])

In [ ]:
sns.lmplot(x='budget', y='revenue', data=scaled_values, fit_reg=False)

In [ ]:
scaled_values = movies_with_revenue[['title', 'budget', 'revenue']].sort_values('budget', ascending=False)[:50]
scaled_values['budget'] = scaled_values['budget'].apply(lambda x: x / 1000000)
scaled_values['revenue'] = scaled_values['revenue'].apply(lambda x: x / 1000000)
# scaled_values.head()
sns.boxplot(data=scaled_values[['budget']])

In [ ]:
# Suppose we want to have an idea of what sort of distribution budgets have in our movie dataset
sns.distplot(movies_with_revenue[['budget']].apply(lambda x: x / 1000000))
# plt.xlim(0, max(movies_with_revenue['budget'].apply(lambda x: x / 1000000)))

### Matplotlib

In [ ]:
# Prepare data by sorting and splitting into two separate entities per axii
budgets = movies_with_revenue['budget'].apply(lambda x: x / 1000000).tolist()
revenues = movies_with_revenue['revenue'].apply(lambda x: x / 1000000).tolist()

# Monstrosity to be seen later.
#budgets, revenues = zip(*sorted(zip(movies_with_revenue['budget'].apply(lambda x: x / 1000000).tolist(), movies_with_revenue['revenue'].apply(lambda x: x / 1000000).tolist()), key=lambda budget: budget[0], reverse=True))


plt.axis([0, max(budgets), 0, max(revenues)])
plt.plot(budgets, revenues)
plt.xlabel('Budgets')
plt.ylabel('Revenues')
plt.title('Revenues vs Budgets')
plt.draw()

In [ ]:
plt.axis([0, max(budgets), 0, max(revenues)])
plt.scatter(budgets, revenues)
# We can make this more readable by shrinking down the sizes
#plt.scatter(budgets, revenues, s=2)
plt.xlabel('Budgets')
plt.ylabel('Revenues')
plt.title('Revenues vs Budgets')
#plt.gca().set_aspect('equal', adjustable='box')
plt.draw()

### Revenue by Genre

Let's try to correlate genres and revenues! As each movie has multiple genre, we'll take the simple approach of duplicating the movie for each genre it exists in.

In [ ]:
list(movies_with_revenue['genres'][:1])

In [ ]:
import json

def genres_to_sets(json_string):
    input_list = json.loads(json_string)
    return {genre['name'] for genre in input_list}

genres_to_sets(movies_with_revenue['genres'][19995])

In [ ]:
movies_by_genre = []

for row in movies_with_revenue[['title', 'genres', 'budget', 'revenue', 'ROI']].itertuples():
    for genre in genres_to_sets(row.genres):
        movies_by_genre.append(row._asdict())
        movies_by_genre[-1]['genres'] = genre
        movies_by_genre[-1]['budget'] /= 1000000
        movies_by_genre[-1]['revenue'] /= 1000000
        
movies_by_genre = pd.DataFrame(movies_by_genre)
movies_by_genre.rename(columns={'genres': 'genre'}, inplace=True)
movies_by_genre.head()

In [ ]:
sns.set_style('whitegrid')
plt.rcParams["figure.figsize"] = (20,5)  # Changes the size of the graph
sns.boxplot(x='genre', y='revenue', data=movies_by_genre[movies_by_genre['revenue'] < 1000])

In [ ]:
sns.set_style('whitegrid')
plt.rcParams["figure.figsize"] = (20,5)  # Changes the size of the graph
sns.boxplot(x='genre', y='budget', data=movies_by_genre[movies_by_genre['budget'] < 150])

In [ ]:
sns.set_style('whitegrid')
plt.rcParams["figure.figsize"] = (20,5)  # Changes the size of the graph
sns.boxplot(x='genre', y='ROI', data=movies_by_genre[movies_by_genre['ROI'] < 20000])
#sns.boxplot(x='genre', y='ROI', data=movies_by_genre[movies_by_genre['ROI'] < 2000])
#sns.boxplot(x='genre', y='ROI', data=movies_by_genre[movies_by_genre['ROI'] < 100])
# sns.boxplot(x='genre', y='ROI', data=movies_by_genre[movies_by_genre['ROI'] < 50])

In [ ]:
movies_by_genre[['genre', 'ROI']][movies_by_genre['ROI'] < 1000].groupby('genre').describe()